In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, RocCurveDisplay

import seaborn as sns 
import kagglehub

In [2]:
df=pd.read_csv('/kaggle/input/ingresos-pacientes/ingresos pacientes  - ingresos.csv') 


df.head()

,Número de Paciente,DNI,Nombre del Paciente,Edad,Tipo de Paciente,Fecha de Cita,Cobertura Médica,Médico,Especialidad,Tipo de Consulta,...,Método de Pago,Diagnóstico Principal,Código CIE-10,Tratamiento,Sucursal,Barrio,Año de cita,Fecha de Cita.1,Mes de cita,tipo tratamiento numerico
0,1001,23456789,Mateo Gómez,8,Pediátrico,20/7/2025,OSDE,Ana Gómez,Pediatría,Control,...,Seguro Médico,Crecimiento,R62.9,Vitaminas,Centro,Villa Devoto,2025,20/7/2025,Julio,1
1,1002,12345678,Sofía Ramirez,54,Adulto,21/7/2025,Swiss Medical,Juan Pérez,Cardiología,Rutina,...,Tarjeta,Hipertensión,I10,Medicacón,Norte,Palermo,2025,21/7/2025,Julio,1
2,1003,34567890,Diego Torres,32,Adulto,21/7/2025,Particular,Laura Díaz,Dermatología,Emergencia,...,Efectivo,Erupción,L29.9,Crema tópica,Sur,Belgrano,2025,21/7/2025,Julio,2
3,1004,12345678,Sofía Ramirez,54,Adulto,22/7/2025,Swiss Medical,Juan Pérez,Cardiología,Rutina,...,Seguro Médico,Chequeo General,Z00.0,Ninguno,Norte,Palermo,2025,22/7/2025,Julio,4
4,1005,23456789,Mateo Gómez,8,Pediátrico,22/7/2025,OSDE,Ana Gómez,Pediatría,Control,...,Tarjeta,Vacunación,Z28.3,Vacuna,Centro,Villa Devoto,2025,22/7/2025,Julio,3


In [3]:
# Convertir la columna de fecha a formato datetime
df['Fecha de Cita'] = pd.to_datetime(df['Fecha de Cita'], format='%d/%m/%Y')

# Crear una columna de trimestre
df['Trimestre'] = df['Fecha de Cita'].dt.quarter

# Agrupar y contar las consultas por trimestre
consultas_por_trimestre = df.groupby('Trimestre').size().reset_index(name='Cantidad de Consultas')

In [4]:
df.head(4)

,Número de Paciente,DNI,Nombre del Paciente,Edad,Tipo de Paciente,Fecha de Cita,Cobertura Médica,Médico,Especialidad,Tipo de Consulta,...,Diagnóstico Principal,Código CIE-10,Tratamiento,Sucursal,Barrio,Año de cita,Fecha de Cita.1,Mes de cita,tipo tratamiento numerico,Trimestre
0,1001,23456789,Mateo Gómez,8,Pediátrico,2025-07-20,OSDE,Ana Gómez,Pediatría,Control,...,Crecimiento,R62.9,Vitaminas,Centro,Villa Devoto,2025,20/7/2025,Julio,1,3
1,1002,12345678,Sofía Ramirez,54,Adulto,2025-07-21,Swiss Medical,Juan Pérez,Cardiología,Rutina,...,Hipertensión,I10,Medicacón,Norte,Palermo,2025,21/7/2025,Julio,1,3
2,1003,34567890,Diego Torres,32,Adulto,2025-07-21,Particular,Laura Díaz,Dermatología,Emergencia,...,Erupción,L29.9,Crema tópica,Sur,Belgrano,2025,21/7/2025,Julio,2,3
3,1004,12345678,Sofía Ramirez,54,Adulto,2025-07-22,Swiss Medical,Juan Pérez,Cardiología,Rutina,...,Chequeo General,Z00.0,Ninguno,Norte,Palermo,2025,22/7/2025,Julio,4,3


In [5]:
from sklearn.linear_model import LinearRegression
import numpy as np

X = consultas_por_trimestre[['Trimestre']]
y = consultas_por_trimestre['Cantidad de Consultas']

modelo = LinearRegression()
modelo.fit(X, y)

LinearRegression()

In [6]:
# Se utiliza 5  ya que el ultimo trimestre registrado fue el 4, (oct, nov, dic)
trimestre_futuro = np.array([[5]]) # El próximo trimestre
prediccion = modelo.predict(trimestre_futuro)
print(f"Predicción de consultas para el próximo trimestre: {int(prediccion[0])}")

Predicción de consultas para el próximo trimestre: 113


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [7]:
df_filtrado = df[df['tipo tratamiento numerico'].isin([1, 2])].copy()